### API Calls with TMDB

In [1]:
import os, time,json
from tqdm.notebook import tqdm_notebook
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_ratings_final.csv.gz',
 'title_basics_final.csv.gz',
 'title.basics_real.tsv.gz',
 'title.akas_real.csv',
 'title_akas_final.csv.gz',
 '.ipynb_checkpoints',
 'title.akas_real.tsv.gz',
 'title.ratings_real.tsv.gz']

In [2]:
with open('/Users/kass/secret/tmbd_api.json','r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY = login['api-key']

In [4]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # Save the .info and .release dictionaries
    info = movie.info()
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1'] == 'US':
            ## save a "certification key" in info with the certification
            info['certification'] = c['certification']
    
    return info

In [5]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [6]:
import pandas as pd
# Load in the dataframe from part 1
basics = pd.read_csv("Data/title_basics_final.csv.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [7]:
#years we want to explore
YEARS_TO_GET = [2000,2001]

In [8]:
# place to store errors
errors = [ ]

In [9]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()

    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)

    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])


    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"),compression='gzip', index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1459 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1577 [00:00<?, ?it/s]

In [10]:
print(f"- Total errors: {len(errors)}")

- Total errors: 3036


In [12]:
errors

[['tt0113026',
  requests.exceptions.HTTPError('401 Client Error: Unauthorized for url: https://api.themoviedb.org/3/movie/tt0113026?api_key=119a845fd2aad3332bec2f')],
 ['tt0113092',
  requests.exceptions.HTTPError('401 Client Error: Unauthorized for url: https://api.themoviedb.org/3/movie/tt0113092?api_key=119a845fd2aad3332bec2f')],
 ['tt0115937',
  requests.exceptions.HTTPError('401 Client Error: Unauthorized for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=119a845fd2aad3332bec2f')],
 ['tt0116391',
  requests.exceptions.HTTPError('401 Client Error: Unauthorized for url: https://api.themoviedb.org/3/movie/tt0116391?api_key=119a845fd2aad3332bec2f')],
 ['tt0116628',
  requests.exceptions.HTTPError('401 Client Error: Unauthorized for url: https://api.themoviedb.org/3/movie/tt0116628?api_key=119a845fd2aad3332bec2f')],
 ['tt0118694',
  requests.exceptions.HTTPError('401 Client Error: Unauthorized for url: https://api.themoviedb.org/3/movie/tt0118694?api_key=119a845fd2aad3332be

In [11]:
#see if file holds data
df_final = pd.read_csv("Data/final_tmdb_data_2000.csv.gz")
df_final.head()

,Unnamed: 0,imdb_id
0,0,0
